# Movie Review Sentiment Labeling
 - 변수이름짓기


In [1]:
# 판타지 영화들의 평점 평균 계산
import requests
from bs4 import BeautifulSoup
import numpy as np


# 리뷰 개수 리스트 만들기
def GetReviewNum(movie_code_list):
    review_num = []

    for code in movie_code_list:   
        review_page_url = f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&page=1"
        res = requests.get(review_page_url)
        soup = BeautifulSoup(res.text)

        # 리뷰 개수를 가져온다
        num = soup.find('strong', attrs={'class':'total'}).find('em').text

        # 7,302 => 7302로 변경해서 리스트에 추가
        review_num.append(int(num.replace(',', '')))
    
    return review_num


# 영화 리스트 생성 함수 (리턴타입은 {영화 코드: (영화 제목, 평점, 리뷰개수)} 형식의 딕셔너리)
def MakeMovieList(genre_num):

    date = 20220428     # 4월 28일 기준 영화랭킹
    movie_title = []
    movie_code = []
    movie_point = []
    review_num = []
    page_num = 1
    exist_next_page = True
    while(exist_next_page):
        movie_list_url = f"https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&date={date}&tg={genre_num}&page={page_num}"
        
        res = requests.get(movie_list_url)
        soup = BeautifulSoup(res.text, 'lxml')
        
        # 1) 영화 코드 리스트, 영화 제목 리스트 만들기
        element = soup.find_all('div', attrs={'class':'tit5'})
        duplicate_code = False
        for movie in element:

            # 코드 리스트 추가(중복 체크 포함)
            temp_url = movie.find('a')['href']
            for i, n in enumerate(temp_url):
                if n == '=':
                    code = temp_url[i+1:]
                    if code in movie_code:
                        exist_next_page = False
                        duplicate_code = True
                        break
                    else:
                        movie_code.append(code)
            if duplicate_code:
                break

            # 제목 리스트 추가
            title = movie.text.strip()
            movie_title.append(title)

        print(movie_title)

        # 2) 평점 리스트 만들기
        point_list = soup.find_all('td', attrs={'class':'point'})
        for i in range(len(point_list)):
            movie_point.append(point_list[i].text)

        # 3) 리뷰 개수 리스트 만들기
        review_num = GetReviewNum(movie_code)
            
        page_num += 1

    # 4) 영화 코드, 제목, 평점, 리뷰 개수를 합쳐 하나의 딕셔너리로 만들기
    movie_list = {}
    for code, title, point, num in zip(movie_code, movie_title, movie_point, review_num):
        movie_list[int(code)] = (title, float(point), num)

    return movie_list


movie_ls = MakeMovieList(genre_num=2)   # 판타지: 2
print(movie_ls)

['반지의 제왕: 왕의 귀환', '해리 포터와 마법사의 돌', '반지의 제왕: 두 개의 탑', '반지의 제왕: 반지 원정대', '천녀유혼', '가위손', '캐리비안의 해적 - 블랙 펄의 저주', '조 블랙의 사랑', '온워드: 단 하루의 기적', '천녀유혼 3 - 도도도', '벤자민 버튼의 시간은 거꾸로 간다', '천녀유혼 2 - 인간도', '캐리비안의 해적 - 망자의 함', '해리 포터와 비밀의 방', '팀 버튼의 크리스마스 악몽', '스타더스트', '드라큐라', '찰리와 초콜릿 공장', '미녀와 야수', '해리 포터와 아즈카반의 죄수', '스트레인저 댄 픽션', '당신, 거기 있어줄래요', '큐브', '나니아 연대기 - 캐스피언 왕자', '천국의 우편배달부', '13층', '천년여우 여우비', '스파이더위크가의 비밀', '록키 호러 픽쳐 쇼', '호빗: 다섯 군대 전투', '호빗 : 스마우그의 폐허', '아더와 미니모이: 제1탄 비밀 원정대의 출정', '미스 페레그린과 이상한 아이들의 집', '호빗 : 뜻밖의 여정', '존 말코비치 되기', '오늘 밤, 로맨스 극장에서', '불청객', '헝거게임: 캣칭 파이어', '비밀', '판의 미로 - 오필리아와 세 개의 열쇠', '내안의 그놈', '말레피센트', '인랑', '캐리비안의 해적 - 세상의 끝에서', '경계선', '슬리피 할로우', '신비한 동물사전', '메리 포핀스 리턴즈', '케이트 앤 레오폴드', '환생']
['반지의 제왕: 왕의 귀환', '해리 포터와 마법사의 돌', '반지의 제왕: 두 개의 탑', '반지의 제왕: 반지 원정대', '천녀유혼', '가위손', '캐리비안의 해적 - 블랙 펄의 저주', '조 블랙의 사랑', '온워드: 단 하루의 기적', '천녀유혼 3 - 도도도', '벤자민 버튼의 시간은 거꾸로 간다', '천녀유혼 2 - 인간도', '캐리비안의 해적 - 망자의 함', '해리 포터와 비밀의 방', '팀 버튼의 크리스마스 악몽', '스타더스트', '드라큐라', '찰리와 초콜릿 공장', '미녀와

In [2]:
# 미녀와 야수, 천녀유혼은 중복
print(len(movie_ls))    

113


In [3]:
# 9.38 * np.log(7302) + 93.6 * np.log(4329) + ...

# 1. 특정영화리뷰개수 / 총리뷰개수
# 

def Calculate(movie_list):
    rating_sum = 0
    num_of_review = 0
    for key, value in movie_list.items():
        rating_sum += value[1]
        num_of_review += value[2]
    avg_rating = rating_sum / len(movie_list)

    return avg_rating

mean_rating = Calculate(movie_ls)
print(mean_rating)

7.66132743362832


In [4]:
# pickle로 movie_list를 저장
import pickle

PATH = '/content/drive/MyDrive/final-project/data'
with open(PATH + '/movie_list_wb_4.pkl', 'wb') as f:
    pickle.dump(movie_ls, f, protocol=4)

## 리뷰 데이터 Sentiment 입력
 - score 8이상 => 1(Positive)
 - socre 8미만 => 0(Negative)

In [5]:
import pandas as pd

PATH = '/content/drive/MyDrive/final-project/data'
df = pd.read_csv(PATH + '/movie_crawling_판타지.csv')

df.head()

,Unnamed: 0,genre,movie_name,score,review
0,0,판타지,반지의 제왕: 왕의 귀환,10,내가 극장에서 이 영화를 봣다는게 너무 자랑스럽고 감동적입니다
1,1,판타지,반지의 제왕: 왕의 귀환,10,현존하는 최고의 판타지영화... 퀄리티가 절대 애들용이 아님
2,2,판타지,반지의 제왕: 왕의 귀환,10,왜 이영화를이제서야봤는지ㅠㅠ 일편부터연달아보는데 재밌다가도 시간가는게너무안타까웠음ㅠ...
3,3,판타지,반지의 제왕: 왕의 귀환,10,몇년이 지난 영화인데도 소름돋는영화.
4,4,판타지,반지의 제왕: 왕의 귀환,10,반지의제왕 안본사람은 있어도한번본 사람은 없다는게 사실인가요


In [24]:
# Sentiment열 추가
df['Sentiment'] = 2

# score 8점이상이면 1, 미만이면 0
df['Sentiment'] = np.where(df['score'] >= 8, 1, 0)

df.head()

,Unnamed: 0,genre,movie_name,score,review,Sentiment
0,0,판타지,반지의 제왕: 왕의 귀환,10,내가 극장에서 이 영화를 봣다는게 너무 자랑스럽고 감동적입니다,1
1,1,판타지,반지의 제왕: 왕의 귀환,10,현존하는 최고의 판타지영화... 퀄리티가 절대 애들용이 아님,1
2,2,판타지,반지의 제왕: 왕의 귀환,10,왜 이영화를이제서야봤는지ㅠㅠ 일편부터연달아보는데 재밌다가도 시간가는게너무안타까웠음ㅠ...,1
3,3,판타지,반지의 제왕: 왕의 귀환,10,몇년이 지난 영화인데도 소름돋는영화.,1
4,4,판타지,반지의 제왕: 왕의 귀환,10,반지의제왕 안본사람은 있어도한번본 사람은 없다는게 사실인가요,1


In [25]:
print(len(df))
print(len(df[df['score'] >= 8]))
print(len(df[df['Sentiment'] == 1]))
print(len(df[df['score'] < 8]))
print(len(df[df['Sentiment'] == 0]))

88274
58648
58648
29626
29626


In [29]:
df.to_csv(PATH + "/movie_data.csv")